<a href="https://colab.research.google.com/github/Bayrkdartech/Binder/blob/main/Practic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:


# استيراد المكتبات
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from google.colab import drive
import os
import shutil

# 1. توصيل Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. إنشاء مجلد للكاجل داخل Colab
!mkdir -p ~/.kaggle

# 3. نسخ ملف kaggle.json من Google Drive إلى مجلد الكاجل
# غيّر هذا المسار حسب مكان ملف kaggle.json في Drive
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

# 4. ضبط الأذونات
!chmod 600 ~/.kaggle/kaggle.json

# 5. تحميل بيانات من Kaggle
# مثال: تحميل Dataset الطاقة "Hourly Energy Consumption"
!kaggle datasets download -d robikscube/hourly-energy-consumption

# 6. فك الضغط عن البيانات
!unzip hourly-energy-consumption.zip -d data/

# 7. قراءة الملف باستخدام Pandas
import pandas as pd

# مثال: قراءة ملف PJME_hourly.csv
df = pd.read_csv('data/PJME_hourly.csv')

# عرض أول 5 صفوف


##first step ##### we turned the string to datetime

df['Datetime']= pd.to_datetime(df['Datetime'])
df.sort_values('Datetime')
df.set_index('Datetime', inplace=True)
df.resample('h').mean()






print(df.isna().sum())
df.dropna().sum()

# عرض القيم المكررة نفسها
df=df[~df.index.duplicated(keep='first')]
duplicates= df[df.index.duplicated(keep=False)]
print(duplicates)

print(df.isna().sum().sum(),'قيم المفقودة:')


###### --  outliers -- ####


q1= df['PJME_MW'].quantile(0.25)
q3= df['PJME_MW'].quantile(0.75)
iqr= q3 - q1
outliers= df[(df['PJME_MW'] < q1 - 1.5 * iqr) | (df['PJME_MW']> q3 + 1.5 * iqr)]
print(len(outliers),"القيم الشادة")

#عرض اول 10 قيم شادة قبل تعديل
print("قيم الشادة :\n", df.loc[outliers.index,'PJME_MW'].head(10))

#تعديل قيم الشادة
df_modified= df.copy()
median_values= df['PJME_MW'].median()
df_modified.loc[outliers.index,'PJME_MW']=median_values
print('تم تعديل القيم الشادة بواسطة الوسيط')

#عرض قسم الشادة بعد تعديل
print("قيم الشادة :\n", df_modified.loc[outliers.index, 'PJME_MW'].head(10))
# اكتشاف فجوات زمنية
Azez_diff=pd.Timedelta(hours=1)
time_diff= df.index.to_series().diff()
# عرض الفجزات كبيرة
gabs=time_diff[time_diff> Azez_diff]
print(gabs)
# تصنيف الفجوة باستخدام دالة الشركة if
df['get_gap']= time_diff
df['gap_type']= df['get_gap'].apply(lambda x: 'normal' if x== Azez_diff
else ('small gap' if x< pd.Timedelta(days= 1)
else 'larg_gap')

)

#  عرض فجوات كبيرة
larg_gap= df[df['gap_type']== 'larg_gap']
print("الفجوات الزمنية الكبيرة :\n", larg_gap[['PJME_MW', 'get_gap']])
df.head()

# fill the large time gaps
df.index=pd.to_datetime(df.index)

full_index=pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
df= df.reindex(full_index)
df['PJME_MW']= df['PJME_MW'].ffill().bfill()
#print the result after fill in by previous result

print("عدد القيم المفقودة بعد التعبئة  ", df['PJME_MW'].isna().sum())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset URL: https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption
License(s): CC0-1.0
hourly-energy-consumption.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  hourly-energy-consumption.zip
replace data/AEP_hourly.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/AEP_hourly.csv     
  inflating: data/COMED_hourly.csv   
  inflating: data/DAYTON_hourly.csv  
  inflating: data/DEOK_hourly.csv    
  inflating: data/DOM_hourly.csv     
  inflating: data/DUQ_hourly.csv     
  inflating: data/EKPC_hourly.csv    
  inflating: data/FE_hourly.csv      
  inflating: data/NI_hourly.csv      
  inflating: data/PJME_hourly.csv    
  inflating: data/PJMW_hourly.csv    
  inflating: data/PJM_Load_hourly.csv  
  inflating: data/est_hourly.paruqet  
  inflating: data/pjm_hourly_est.csv  
PJME_MW    